# Notebook to update readme files

In [1]:
import re
import toml
import shutil
import posixpath

from pathlib import Path

import pandas as pd

from mintalib import core


In [2]:
ROOTDIR = Path.cwd().parent
PYPROJECT = ROOTDIR.joinpath("pyproject.toml").resolve(strict=True)


In [3]:
def jquery(data: dict, item: str, default=None):
    result = data

    for i in item.split("."):
        result = result.get(i, None)
        if result is None:
            return default

    return result


def get_project_url(pyproject=PYPROJECT):
    """extract project url from project configuration"""

    if not pyproject.exists():
        raise FileNotFoundError("pyproject.toml")

    config = toml.load(pyproject)
    
    return jquery(config, "project.urls.homepage")

get_project_url()


'https://github.com/furechan/mintalib'

In [ ]:

def get_info(func):
    """information about function"""

    info = dict(Name=func.__name__)

    doc = func.__doc__ or ""
    description = doc.strip().partition("\n")[0]
    if description is not None:
        info.update(Description=description)

    return info


def list_indicators():
    """list indicators"""

    from mintalib import indicators

    result = [v for k, v in vars(indicators).items() if k.isupper() and callable(v)]

    result = [get_info(f) for f in result]

    result = pd.DataFrame(result).set_index("Name")

    result = result.sort_index()

    return result

list_indicators()

,Input,Description
Name,,
ABS,Series,Absolute Value
ADX,Prices,Average Directional Index
ALMA,Series,Arnaud Legoux Moving Average
ATR,Prices,Average True Range
AVGPRICE,Prices,Average Price
...,...,...
TSF,Series,Time Series Forecast (linear regression)
TYPPRICE,Prices,Typical Price
UPDOWN,Series,Flag for value crossing up & down levels


In [5]:
from mintalib import indicators

vars(indicators)

{'__name__': 'mintalib.indicators',
 '__doc__': '\nFactory functions for technical analysis indicators.\n\nIndicator factory names are all upper case.\n\nIndicators offer a composable interface where a calculation routine\nis bound together with its calculation parameters.\n\nAn indicator object is a callable that can be applied to prices or series data.\n\nIndicators can be chained with the `@` operator as in `ROC(1) @ SMA(20)`.\n\nThe `@` operator can also be used to apply an indicator to its parameter.\n\nSo for example `SMA(50) @ prices` can be used to compute the 50 period simple moving average on `prices`,\ninstead of the more verbose `SMA(50)(prices)`.\n',
 '__package__': 'mintalib',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x10b71ae00>,
 '__spec__': ModuleSpec(name='mintalib.indicators', loader=<_frozen_importlib_external.SourceFileLoader object at 0x10b71ae00>, origin='/Users/frederic/Projects/mintalib/src/mintalib/indicators.py'),
 '__file__': '/Users/fr

In [6]:
get_info(indicators.EMA)


{'Name': 'EMA', 'Input': 'Series', 'Description': 'Exponential Moving Average'}

In [7]:
indicators.EMA.__name__


'EMA'

In [8]:




indicators = list_indicators()
indicators

,Input,Description
Name,,
ABS,Series,Absolute Value
ADX,Prices,Average Directional Index
ALMA,Series,Arnaud Legoux Moving Average
ATR,Prices,Average True Range
AVGPRICE,Prices,Average Price
...,...,...
TSF,Series,Time Series Forecast (linear regression)
TYPPRICE,Prices,Typical Price
UPDOWN,Series,Flag for value crossing up & down levels


In [9]:
def backup_readme(verbose=True):
    readme = ROOTDIR.joinpath("README.md")
    if ROOTDIR.joinpath("archive").exists():
        backup = ROOTDIR.joinpath("archive/README.bak")
    else:
        backup = ROOTDIR.joinpath("README.bak")

    if verbose:
        print(f"Backing up {readme.name} ...")

    shutil.copy(readme, backup)


backup_readme()

Backing up README.md ...


In [10]:
def update_readme(verbose=True):
    title = "## List of Indicators\n"
    table = list_indicators().to_markdown()
    repl = title + "\n" + table + "\n\n\n"

    # flags m=multiline, s=dotall
    pattern = r"(?ms)(^[#]+ List of (Functions|Indicators)\n[^#]+)"

    readme = ROOTDIR.joinpath("README.md")
    contents = readme.read_text()

    output, count = re.subn(pattern, repl, contents)

    if count != 1:
        raise RuntimeError("Cound not locate list of funtions")

    if verbose:
        print(f"Updating {readme.name} ...")

    return readme.write_text(output)


update_readme()

Updating README.md ...


11046

In [11]:
def process_readme(file, *, project_url=None, branch="main", verbose=True):
    """translate relative urls to full urls"""

    if project_url is None:
        project_url = get_project_url()

    def replace(m):
        exclam, alt, url = m.groups()
        ftype = "raw" if exclam else "blob"
        if url.startswith("/"):
            url = url.removeprefix("/")
            url = posixpath.join(project_url, ftype, branch, url)
            text = f"{exclam}[{alt}]({url})"
            if verbose:
                print("mapping", m.group(0), "->", text)
        else:
            text = m.group(0)
        return text

    source = file.read_text()

    result = re.sub(
        r"(\!?) \[ ([^]]*) \] \( ([^)]+) \)", replace, source,
        flags = re.VERBOSE
        )

    return result



In [12]:
README = ROOTDIR.joinpath("README.md").resolve(strict=True)
OUTDIR = ROOTDIR / "output"
OUTDIR.mkdir(exist_ok=True)

output = process_readme(README, verbose=True)

outfile = OUTDIR.joinpath("pypi-readme.md").resolve()

print(f"Updating {outfile.name} ...")

outfile.write_text(output)

Updating pypi-readme.md ...


11046